# Use LLaMA-Factory to training

In [4]:
# Check GPU configuration
!nvidia-smi

Sun Jul 14 17:00:08 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P8              13W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [5]:
# Install LLaMA Factory dependencies
%cd /content/
%rm -rf LLaMA-Factory
!git clone https://github.com/hiyouga/LLaMA-Factory.git
%cd LLaMA-Factory
%ls
!pip install -e .[torch,bitsandbytes]
!pip install rouge-chinese

/content
Cloning into 'LLaMA-Factory'...
remote: Enumerating objects: 15452, done.
remote: Counting objects: 100% (402/402), done.
remote: Compressing objects: 100% (202/202), done.
remote: Total 15452 (delta 249), reused 316 (delta 199), pack-reused 15050
Receiving objects: 100% (15452/15452), 221.74 MiB | 15.81 MiB/s, done.
Resolving deltas: 100% (11298/11298), done.
/content/LLaMA-Factory
assets/       docker/      LICENSE      pyproject.toml  requirements.txt  src/
CITATION.cff  evaluation/  Makefile     README.md       scripts/          tests/
data/         examples/    MANIFEST.in  README_zh.md    setup.py
Obtaining file:///content/LLaMA-Factory
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for llamafactory (pyproject.toml) ... done
  Created wheel for llamafactory: filename=llamafactory-0.8.3.dev0-0.

In [6]:
# Modify the content of the NAME and AUTHOR variables.
import json

%cd /content/LLaMA-Factory/

NAME = "Wahoo-llama"
AUTHOR = "SXM"

with open("data/identity.json", "r", encoding="utf-8") as f:
  dataset = json.load(f)

for sample in dataset:
  sample["output"] = sample["output"].replace("{{"+ "name" + "}}", NAME).replace("{{"+ "author" + "}}", AUTHOR)

with open("data/identity.json", "w", encoding="utf-8") as f:
  json.dump(dataset, f, indent=2, ensure_ascii=False)

/content/LLaMA-Factory


In [7]:
!pip install --upgrade huggingface_hub
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: write)

In [8]:
# Load the dataset and use it
from datasets import load_dataset
import hashlib

sha1 = hashlib.sha1()
ds = load_dataset("CAS-SIAT-XinHai/CPsyCoun")

dataset_bytes = json.dumps(ds['train'][:]).encode('utf-8')

# Calculate SHA-1 hash value
sha1.update(dataset_bytes)
hash_value = sha1.hexdigest()

print("SHA-1 Hash:", hash_value)

dataset_list = [entry for entry in ds['train']]


with open('/content/LLaMA-Factory/data/psychology_data.json', 'w', encoding='utf-8') as f:
    json.dump(dataset_list, f, ensure_ascii=False, indent=4)

dataset_info_path = '/content/LLaMA-Factory/data/dataset_info.json'
dataset_info = {
    "psychology_data": {
        "file_name": "psychology_data.json",
        "file_sha1": hash_value
    }
}

try:
    with open(dataset_info_path, 'r', encoding='utf-8') as f:
        existing_info = json.load(f)
except FileNotFoundError:
    existing_info = {}

existing_info.update(dataset_info)

with open(dataset_info_path, 'w', encoding='utf-8') as f:
    json.dump(existing_info, f, ensure_ascii=False, indent=4)

print(f"Updated {dataset_info_path} with new dataset info.")

SHA-1 Hash: 96c066885251d4f97be91f7fb9ab11867a0efc74
Updated /content/LLaMA-Factory/data/dataset_info.json with new dataset info.


In [ ]:
# Open the running environment
%cd /content/LLaMA-Factory/
!GRADIO_SHARE=1 llamafactory-cli webui

/content/LLaMA-Factory
2024-07-14 17:01:59.314641: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-14 17:01:59.314704: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-14 17:01:59.316145: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-14 17:01:59.323442: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-14 17:02:00.57

# In order to upload the model to HuggingFace

In [ ]:
!pip install huggingface_hub

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from huggingface_hub import HfApi
api = HfApi()

model_id = "username/fine-tune-model" #@param {type:"string"}
export_folder_path = "/content/LLaMA-Factory/export" #@param {type:"string"}
api.create_repo(model_id, private=True, exist_ok=True, repo_type="model")
api.upload_folder(
    folder_path=export_folder_path,
    repo_id=model_id,
)

# Save in Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
Google_Drive_Folder = "/content/drive/MyDrive/LLaMA-Factory/" #@param {type:"string"}

!cp -rf /content/LLaMA-Factory/saves {Google_Drive_Folder}